In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_rgb_histograms (image, roi):
    r_image, g_image, b_image = image[:,:,0],image[:,:,1],image[:,:,2]
    r_roi, g_roi, b_roi = roi[:,:,0],roi[:,:,1],roi[:,:,2]

    r_image_hist = cv2.calcHist([r_image],[0],None,[256],[0,256])
    g_image_hist = cv2.calcHist([g_image],[0],None,[256],[0,256])
    b_image_hist = cv2.calcHist([b_image],[0],None,[256],[0,256])
    r_roi_hist = cv2.calcHist([r_roi],[0],None,[256],[0,256])
    g_roi_hist = cv2.calcHist([g_roi],[0],None,[256],[0,256])
    b_roi_hist = cv2.calcHist([b_roi],[0],None,[256],[0,256])

    image_color = 'royalblue'
    roi_color = 'goldenrod'
    fig, axis  = plt.subplots(3,1)
    axis[0].plot(r_image_hist, color = image_color)
    axis[0].plot(r_roi_hist, color = roi_color)
    axis[0].set_title('Red distribution')
    axis[1].plot(g_image_hist, color = image_color)
    axis[1].plot(g_roi_hist, color = roi_color)
    axis[1].set_title('Green distribution')
    axis[2].plot(b_image_hist, color = image_color)
    axis[2].plot(b_roi_hist, color = roi_color)
    axis[2].set_title('Blue distribution')
    fig.tight_layout()
    plt.show()

def rgb_mask_roi(lower_tresh, upper_tresh,image):
    lower_thresh = np.array(lower_tresh)
    upper_thresh = np.array(upper_tresh)

    mask_roi = cv2.inRange(image, lower_thresh, upper_thresh)
    masked_image = cv2.bitwise_and(image,image, mask = mask_roi)
    gray_image = cv2.cvtColor(masked_image,cv2.COLOR_RGB2GRAY)
    threshold = cv2.threshold(gray_image, 10,255, cv2.THRESH_BINARY)[1]

    return (masked_image,threshold)

def plot_multiple_images (image1, image2,image3):
    cmap = 'plasma'
    fig = plt.figure(figsize=(10,15))
    gs = fig.add_gridspec(1, 3, hspace=0, wspace=0)
    (ax1, ax2, ax3) = gs.subplots(sharex='col', sharey='row')
    ax1.imshow(image1)
    ax2.imshow(image2)
    ax3.imshow(image3, cmap= cmap)
    for ax in fig.get_axes():
        ax.label_outer()

def display_tracking_window_axis_center(frame,x,y,tw_width,tw_height, rows):
        bounding_box = cv2.rectangle(frame,
                  (x, y),
                  (x + tw_width, y + tw_height),
                  (61,89,171), 2)
        horisontal_axis = cv2.line(frame,
                        (x + int(tw_width/2), 0),
                        (x + int(tw_width/2), rows),
                        (255, 0, 0), 1)
        object_center = cv2.circle(frame,
                                (x + int(tw_width/2), y + int(tw_height/2)),
                                2, (0,0,255),2)
        
        return (bounding_box, horisontal_axis,object_center)

In [ ]:
video = cv2.VideoCapture(r'.\input_data\DJI_0174_cut.mp4')
first_frame = video.read()[1]
plt.imshow(first_frame)

In [ ]:
roi_start_x = 0
roi_end_x = 1080
roi_start_y = 880
roi_end_y = 1240
frame_roi = first_frame[roi_start_x:roi_end_x, roi_start_y:roi_end_y]

Masking using RGB color model

In [ ]:
plot_rgb_histograms(first_frame,frame_roi)

In [ ]:
lower_rgb = [50,50,55]
upper_rgb = [100,100,100]
masked_image, threshold = rgb_mask_roi(lower_rgb, upper_rgb,first_frame)

plot_multiple_images(first_frame,masked_image,threshold)

Masking video and applying tracking window (RGB)

In [ ]:
tw_start_point_x = 750
tw_start_point_y = 300
tw_start_width = 360
tw_start_height = 1000
term_criteria= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,1)

while True:
    frame = video.read()[1]

    frame_roi = frame[roi_start_x:roi_end_x, roi_start_y:roi_end_y]
    masked_image,threshold = rgb_mask_roi(lower_rgb, upper_rgb,frame)
    

    track_window = cv2.meanShift(threshold,
                                 (tw_start_point_x, tw_start_point_y,
                                  tw_start_width, tw_start_height),
                                  term_criteria)[1]
    tw_start_point_x, tw_start_point_y, tw_width, tw_height = track_window
    rows, cols, _bands = frame.shape

    bounding_box, horisontal_axis, object_center = display_tracking_window_axis_center(
        frame,tw_start_point_x, tw_start_point_y,tw_width,tw_height,rows)

    cv2.imshow('Frame', frame)
    key = cv2.waitKey(100) #for framerate 20, normal playback is 100
    if key == 27:
        break
video.release()
cv2.destroyAllWindows()